<a href="https://colab.research.google.com/github/talhadar1/real_estate_price_forecasting/blob/master/real_estat_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# from google.colab import drive
# drive.mount('/content/drive/')
# import os
# os.getcwd()
# !pip install -U pip install pandas-profiling[notebook,html]
pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip


In [0]:
# #Adding more RAM to the COLAB machine:
# #https://towardsdatascience.com/upgrade-your-memory-on-google-colab-for-free-1b8b18e8791d
# a = []
# while(1):
#     a.append('1')

# **Import libraries and files:**

In [0]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve

folder_path = '/content/drive/My Drive/seminar_data/'
file_name = 'TransactionsXLSsince2008.xlsx'

# **import data:**

In [2]:
#import dataset from xlsx file
orig_df = pd.read_excel(folder_path + file_name)
# Success
print("Israel housing dataset has {} data points with {} variables each.".format(*orig_df.shape))

Israel housing dataset has 664606 data points with 18 variables each.


# **Data Exploration:**
In the first section of the project, we will make an exploratory analysis of the dataset and provide some observations.

Initially we could make the following intuitive assumptions for each feature:


*   Property area with a bigger square meters (higher ‘shetachneto’ value) will worth more. Usually Property with more shetachneto  can fit more people, so it is reasonable that they cost more money. They are directly proportional variables.
*   Neighborhoods with lower Socioeconomic Index (lower ‘soccesso’ value) will worth less. If the Socioeconomic Index value is low at a neighborhood , it is likely that they have low purchasing power and therefore, they houses will cost less. They are directly proportional variables.
*  Property built later (higher ‘Year of Construction’ value) will worth more. Usually later built property will have newer construction and will feature more innovative features (elevator, shelter, green building, underground parking, etc.). They are directly proportional variables.

In [3]:
ProfileReport(orig_df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,18
Number of observations,664606
Total Missing (%),5.7%
Total size in memory,91.3 MiB
Average record size in memory,144.0 B
Numeric,12
Categorical,6
Boolean,0
Date,0
Text (Unique),0
Rejected,0




*   tat_helka has a high cardinality: 13916 distinct values Warning

*   date_transaction has a high cardinality: 3825 distinct values Warning

*   city_hebrew has a high cardinality: 784 distinct values Warning

*   rechov has a high cardinality: 13973 distinct values Warning 

---

1.   Note that the transaction date appears as both date_transaction and split date year, month and day. The full date does not give us good data for days and months so we will lower it. And leave the three properties of the second option.
2.  'rechov' - This variable represents the street name, the street name has no empirical significance to the price of the apartment directly and we see that there is no correlation between the street name and the price of the property.  We will drop this property!
3.  'city_hebrew' - This variable represents the name of the city, the name of the city has no empirical significance to the price of the apartment directly and we see that there is no correlation at all between the name of the city and the price of the property. There may be identical apartments with their characteristics in the same city but with different prices. We will drop this property!
4. 'tat_helka' - This variable represents the number of subdivisions where the property is located within the block (usually 2-4 properties),
A sub-plot has no direct empirical significance to the price, but only when it comes in combination with the block and plot where it exists.
We will drop this property!

5. 'bayit' & 'dira' - This variable represents the building number and apartment number on the street where the property is located respectively,
Similarly to the street name, this variable is also not directly correlated with the price, We will drop this properties!


In [0]:
#Drop any unnecessary features
df = orig_df.drop(['rechov', 'date_transaction', 'city_hebrew', 'tat_helka', 'bayit', 'dira'], axis=1).dropna(how='all')

Validate the dimensions of the dataframe:

In [5]:
# dataframe.size - number of cells
size = orig_df.size 
  
# dataframe.shape - shape[0]=number of rows  | shape[1]=number of columns
shape = orig_df.shape 

# printing size and shape 
print("number of cells = {}\nShape ={}\#ROWS x #COLUMNS = {}". format(size, shape, shape[0]*shape[1])) 

number of cells = 11962908
Shape =(664606, 18)\#ROWS x #COLUMNS = 11962908


#Handling missing & duplicate data:
https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779

*   floors has 85099 / 12.8% missing values Missing

*   dirotbnyn has 179834 / 27.1% missing values Missing

*   soceco has 87191 / 13.1% missing values Missing

*   Dataset has 332 duplicate rows Warning

In [0]:
# Drop the 5-th percentile
df = df[df['year']>1950]

# Change all 'floors' with NaN to - 5 (Median of the floors column)
df['floors'] = df['floors'].fillna(5)

# Change all 'dirotbnyn' with NaN to - 5 (Median of the dirotbnyn column)
df['dirotbnyn'] = df['dirotbnyn'].fillna(18)

# Drop all rows with NaN at 'soceco'
df['soceco'] = df['soceco'].fillna(0.16788)
# df['soceco'] = df['soceco'].dropna(how='all')

# drop all duplicate rows in the dataframe
df.drop_duplicates(keep=False,inplace=True) 


This cell is exuted only because of the lack of hardware resources on the machine we run the project on.

In [0]:
#take only precent of the data set for training and testing. 
data_set_precent = 0.7
df = df.sample(frac=data_set_precent)

In the variable "gush_helka" there are 13916 different values ​​by dropping the "helka" we can reduce by 50%. Because this variable was a categorical variable we would need every value in which the dummy variable and therefore this reduction would improve the runtime performance of the model.

if the value was "002023-432" it will be "002023" after this cell execution.

In [0]:
df['gush'] = df['gush_helka'].apply(lambda x: x.split('-')[0])
df = df.drop('gush_helka', axis=1)
df.head()

,price_sheqel,year_of_construction,floor,shetachneto,type,floors,dirotbnyn,year,month,day,soceco,gush
479630,1340000.0,2000.0,4.0,99.0,דירה בבית קומות,6.0,18.0,2011.0,5.0,19.0,1.040512,003703
159200,2000000.0,2016.0,1.0,162.0,דירה בבית קומות,8.0,18.0,2016.0,11.0,17.0,0.195855,038268
55965,1280000.0,2001.0,0.0,121.0,קוטג' דו-משפחתי,2.0,1.0,2018.0,10.0,21.0,-0.492229,002843
442314,1765000.0,2005.0,0.0,178.0,קוטג' דו-משפחתי,5.0,2.0,2012.0,1.0,3.0,0.167880,073750
42058,500000.0,1970.0,3.0,58.0,דירה בבית קומות,4.0,22.0,2018.0,12.0,21.0,-0.329497,017746


In [10]:
ProfileReport(df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,13
Number of observations,457940
Total Missing (%),0.0%
Total size in memory,45.4 MiB
Average record size in memory,104.0 B
Numeric,11
Categorical,2
Boolean,0
Date,0
Text (Unique),0
Rejected,0


**Dataset It has no missing values ​​and no duplicates rows**


'gush' and 'type' both are **categorical variables**.

There is no empirical meaning to price proportionally. If one neighborhood number is '7000' and a second neighborhood number is '9000' this does not mean that in the second neighborhood property prices are higher than in the first neighborhood and vice versa.

Similarly, if the type of property we were an apartment building does not mean that its price will be higher or lower than the price of a private house.

**Therefore we will make these variables a dummy variable.**

In [11]:
df = pd.get_dummies(df.dropna())
df.head()

# #Creating the same dataframe without the socioeconomic property of the property.
# df_2 = df.drop('soceco', axis=1)

,price_sheqel,year_of_construction,floor,shetachneto,floors,dirotbnyn,year,month,day,soceco,type_בית בודד,type_דירה בבית קומות,type_דירת גג,type_דירת גן,type_דירת דופלקס,type_קוטג' דו-משפחתי,type_קוטג' חד-משפחתי,type_קוטג' טורי,gush_000001,gush_000071,gush_000117,gush_000119,gush_000124,gush_000182,gush_000187,gush_000204,gush_000240,gush_000259,gush_000265,gush_000266,gush_000268,gush_000283,gush_000301,gush_000309,gush_000310,gush_000311,gush_000326,gush_000351,gush_000374,gush_000390,...,gush_201046,gush_201047,gush_202000,gush_202009,gush_202011,gush_204003,gush_204005,gush_204006,gush_204007,gush_204008,gush_204011,gush_204015,gush_204017,gush_204025,gush_204026,gush_204028,gush_204101,gush_204301,gush_204304,gush_204503,gush_204551,gush_204701,gush_204702,gush_215001,gush_220000,gush_242002,gush_246001,gush_260000,gush_279001,gush_400175,gush_400403,gush_400495,gush_400505,gush_401006,gush_401009,gush_401022,gush_691800,gush_692400,gush_900006,gush_900009
479630,1340000.0,2000.0,4.0,99.0,6.0,18.0,2011.0,5.0,19.0,1.040512,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
159200,2000000.0,2016.0,1.0,162.0,8.0,18.0,2016.0,11.0,17.0,0.195855,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55965,1280000.0,2001.0,0.0,121.0,2.0,1.0,2018.0,10.0,21.0,-0.492229,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
442314,1765000.0,2005.0,0.0,178.0,5.0,2.0,2012.0,1.0,3.0,0.167880,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
42058,500000.0,1970.0,3.0,58.0,4.0,22.0,2018.0,12.0,21.0,-0.329497,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


dataset has now 6322 different variables(columns).
10 of them are original and the other are dummy variables
  

#Developing a Model#
Defining a Performace Metric:
we will calculate the coefficient of determination, R², to quantify the model’s performance. The coefficient of determination for a model is a useful statistic in regression analysis, as it often describes how “good” that model is at making predictions.

In [0]:
# Import 'r2_score'

from sklearn.metrics import r2_score

def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true (y_true) and predicted (y_predict) values based on the metric chosen. """
    
    score = r2_score(y_true, y_predict)
    
    # Return the score
    return score

##Shuffle and Split Data:




In [20]:
#Tests data is acording to the 'Pareto principle' is  80% of the data is for training and the rest 20% for testing
#https://en.wikipedia.org/wiki/Pareto_principle
test_frac = .2 

#(1) Generating dataframes for training and testing:
train_df, test_df, y_train, y_test = train_test_split(df.drop('price_sheqel', axis=1), df['price_sheqel'], test_size=test_frac,random_state = 42)

#(2) Generating dataframes for training and testing without the 'soceco' feature:
train_df2, test_df2, y_train2, y_test2 = train_test_split(df_2.drop('price_sheqel', axis=1), df_2['price_sheqel'], test_size=test_frac,random_state = 42)

#We don't need to save the original dataframes because we already split them. Freeing them from memory will contribute to program performance.
# del df , df_2
print("Training and testing split was successful.")

Training and testing split was successful.


##Training and Testing#

### **Random Forest Regressor :**

In [0]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

#Each leaf will contain at least 'min_samples_in_a_leaf' sampels. it will help us control the depth of the trees.
min_samples_in_a_leaf = 30

In [0]:
# Instantiate 2 models with 100 decision trees each

rf = RandomForestRegressor(n_estimators = 100, random_state = 42, verbose=100, min_samples_leaf=min_samples_in_a_leaf, n_jobs=-1)
rf_no_soceco = RandomForestRegressor(n_estimators = 100, random_state = 42, verbose=100, min_samples_leaf=min_samples_in_a_leaf, n_jobs=-1)


####Train the model on training data


In [0]:
#Train the Random Forest models on training data:
rf.fit(train_df, y_train)
rf_no_soceco.fit(train_df2, y_train2)

####Test the model on the the test data

In [0]:
# Use the forest's predict method on the test data
predictions  = rf.predict(test_df)
predictions_no_soceco = rf_no_soceco.predict(test_df2)

# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print(f'Mean Absolute Error: {round(np.mean(errors), 2)} shekel')
print(f'Median Absolute Error: {round(np.median(errors), 2)} shekel')

# Calculate the absolute errors
errors = abs(predictions_no_soceco - y_test2)
# Print out the mean absolute error (mae)
print(f'Predictions_no_soceco mean Absolute Error: {round(np.mean(errors), 2)} shekel')
print(f'Predictions_no_soceco median Absolute Error: {round(np.median(errors), 2)} shekel')

####Save model to file:

In [0]:
import joblib
joblib_file =folder_path + 'RandomForestRegressor_model.pkl'
joblib_file_no_soceco =folder_path + 'RandomForestRegressor_model_no_soceco.pkl'
joblib.dump(rf,joblib_file)
joblib.dump(rf_no_soceco,joblib_file_no_soceco)

# test_df.to_csv (folder_path+ r'export_dataframe.csv', index = False, header=True)
# y_test.to_csv  (folder_path + r'export_dataframe.csv', index = False, header=True)
dataframe_row = folder_path + 'dataframe_row.csv'
test_df.head(1).to_csv (dataframe_row, index = False, header=True)


#### Load model from file:

In [0]:
loaded_model = joblib.load(joblib_file)


In [0]:
test_index = 3
input_row = test_df.head(test_index)
real_output = y_test.head(test_index)
print(input_row )
print(real_output)
prediction = loaded_model.predict(input_row)
error = abs(prediction - y_test.head(test_index))
print(f'Prediction Error: {error} shekel')

# prediced_row.head()

In [0]:
test_index = 3
input_row = test_df.iloc[[test_index]]
print (input_row)
real_output = y_test.iloc[[test_index]]
print(input_row )
print(real_output)
prediction = loaded_model.predict(input_row)
error = abs(prediction - y_test.iloc[[test_index]])
print(f'Prediction Error: {error} shekel')
print(error)


In [0]:
importances = list(zip(train_df.columns.to_list(), rf.feature_importances_))
epsilon = 0.0001
count = 0
for col, val in sorted(importances, key=lambda x: x[1], reverse=True):
    if val > epsilon:
      count = count + 1
      print(f'{col}: {val}')
print(count)

In [0]:
# original.dropna()[original['gush_helka'].dropna().str.contains('009126')].sort_values('price_sheqel')

###**DecisionTreeRegressor**

In [0]:
# Import 'make_scorer', 'DecisionTreeRegressor', and 'GridSearchCV'
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.20, random_state = 0)

    # Create a decision tree regressor object
    regressor = DecisionTreeRegressor()

    # Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {'max_depth':[1,2,3,4,5,6,7,8,9,10]}

    # Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(performance_metric)

    # Create the grid search cv object --> GridSearchCV()
    grid = GridSearchCV(estimator=regressor, param_grid=params, scoring=scoring_fnc, cv=cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

####Optimal Model
The following code snippet finds the maximum depth that return the optimal model.

In [0]:
# Fit the training data to the model using grid search
# del train_df['type']
train_df.dropna().head()
reg = fit_model(train_df, y_train)

# Produce the value for 'max_depth'
# print("Parameter 'max_depth' is {} for the optimal model.".format(reg.get_params()['max_depth']))


### **Linear Regression**

In [0]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(train_df, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
#Predict method on the test data
predictions = reg.predict(test_df)

In [0]:
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print(f'Mean Absolute Error: {round(np.mean(errors), 2)} shekel')

## **SGD Regression**

In [0]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor()
sgd_reg.fit(train_df, y_train)

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [0]:
#Predict method on the test data
predictions = sgd_reg.predict(test_df)

In [0]:
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print(f'Mean Absolute Error: {round(np.mean(errors), 2)} shekel')

Mean Absolute Error: 181392209066565.28 shekel


In [0]:
test_df.head(2)
sgd_reg.predict(test_df.head(2))

array([1.86075595e+14, 1.86528809e+14])